In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()

# Layer 1: Convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Layer 2: Convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Layer 3: Convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())

# Fully connected layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer (7 emotions)
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = 'archive (2)/train'
test_dir = 'archive (2)/test'

# Normalize and load data using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values (0-1)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # Images resized to 48x48 pixels
    batch_size=64,
    class_mode='categorical',
    color_mode='grayscale'  # Grayscale images
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    color_mode='grayscale'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,  # Number of epochs can be adjusted
    validation_data=test_generator
)

# Save the trained model
model.save('emotion_detection_model.h5')


Epoch 1/25
449/449 [==============================] - 366s 809ms/step - loss: 1.7704 - accuracy: 0.2754 - val_loss: 1.6455 - val_accuracy: 0.3703
Epoch 2/25
449/449 [==============================] - 90s 201ms/step - loss: 1.5811 - accuracy: 0.3820 - val_loss: 1.4599 - val_accuracy: 0.4469
Epoch 3/25
449/449 [==============================] - 78s 173ms/step - loss: 1.4624 - accuracy: 0.4340 - val_loss: 1.3544 - val_accuracy: 0.4822
Epoch 4/25
449/449 [==============================] - 73s 162ms/step - loss: 1.3969 - accuracy: 0.4611 - val_loss: 1.3232 - val_accuracy: 0.4886
Epoch 5/25
449/449 [==============================] - 76s 168ms/step - loss: 1.3494 - accuracy: 0.4807 - val_loss: 1.2783 - val_accuracy: 0.5100
Epoch 6/25
449/449 [==============================] - 75s 168ms/step - loss: 1.3119 - accuracy: 0.5007 - val_loss: 1.2433 - val_accuracy: 0.5128
Epoch 7/25
449/449 [==============================] - 76s 168ms/step - loss: 1.2802 - accuracy: 0.5149 - val_loss: 1.2132 - val_a

In [5]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


113/113 [==============================] - 21s 183ms/step - loss: 1.0824 - accuracy: 0.5957
Test Loss: 1.0824378728866577
Test Accuracy: 0.5957090854644775


In [9]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load a test image
image_path = 'div.jpg'  # Replace with an actual image path
img = load_img(image_path, target_size=(48, 48), color_mode='grayscale')  # FER2013 images are grayscale
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict emotion
prediction = model.predict(img_array)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
predicted_emotion = emotion_labels[np.argmax(prediction)]

# Display result
print(f"Predicted Emotion: {predicted_emotion}")


1/1 [==============================] - 0s 42ms/step
Predicted Emotion: Happy
